In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    # si usan colab, deben cambiar el token de esta url
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/MetObjects_sample.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/MetObjects_sample.csv?token=AA4GFHOEXS4SBI2SSURWCAK6WSDEW
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

[Dataset](#section_dataset)

[Imports](#section_imports)

[Tidy data](#section_tidy_data)

[Repaso de joins con pandas](#section_repaso_joins)

[Referencias](#section_referencias)


---


# Pandas 2 - tidy data

<a id="section_intro"></a> 
## Intro

[volver a TOC](#section_toc)


Decimos que un dataset está ordenado cuando:
* Cada variable es una columna
* Cada observación es una fila
* Cada tipo de unidad observacional forma una tabla

Algunas definiciones:
* Variable: Es la medición de un atributo, por ejemplo, peso, altura, etc 
* Valor: Es la medida que toma una variable para una observación
* Observación: Todas las observaciones toman el mismo tipo de valores para cada variable. 



<a id="section_dataset"></a> 
## Dataset

[volver a TOC](#section_toc)


Usaremos el dataset de las obras del Met (The Metropolitan Museum of Art) que vimos en la clase de Limpieza de datos

https://github.com/metmuseum/openaccess/

Analizando el dataset del Met vemos que los datos de los artistas están en la misma tabla que los datos de las obras, incumpliendo la tercer regla que dice "Cada tipo de unidad observacional forma una tabla".

Veamos cómo resolver este punto.


<a id="section_imports"></a> 
## Imports

[volver a TOC](#section_toc)


In [ ]:
import pandas as pd
import numpy as np

## Importamos los datos

Vemos de qué tamaño es el dataset e imprimimos losprimeros registros

In [ ]:
# local
data_location = "../Data/MetObjects_sample.csv"

data = pd.read_csv(data_location)

data.dtypes

In [ ]:
data.head(3)

<a id="section_tidy_data"></a> 
## Tidy data

[volver a TOC](#section_toc)

Analizando el dataset del Met vemos que los datos de los artistas están en la misma tabla que los datos de las obras, incumpliendo la tercer regla que dice "Cada tipo de unidad observacional forma una tabla".

En este caso deberíamos tener una tabla para Obra y otra para Artista.

Separemos estos datos en dos tablas, y eliminemos duplicados en la tabla de artistas.

In [ ]:
data_artist_columns = ['Artist Role', 'Artist Prefix', 'Artist Display Name',
       'Artist Display Bio', 'Artist Suffix', 'Artist Alpha Sort',
       'Artist Nationality', 'Artist Begin Date', 'Artist End Date',
       'Artist Gender', 'Artist ULAN URL', 'Artist Wikidata URL']

data_artist  = data.loc[:, data_artist_columns]
print(data_artist.shape)


In [ ]:
data_artist.head(3)

Veamos si hay registros duplicados en data_artist. 

Consideramos duplicados aquellos registros que coincidan en los campos "Artist Display Name", "Artist Role"

In [ ]:
artist_duplicated = data_artist.duplicated(subset=["Artist Display Name", "Artist Role"])
any(artist_duplicated)

Eliminamos los registros duplicados:

In [ ]:
data_artist_unique = data_artist.drop_duplicates(subset=["Artist Display Name", "Artist Role"], keep="first")
print(data_artist_unique.shape)
print(data_artist.shape)

Para no perder la asociación entre la obra y el artista, en la tabla data_object mantenemos los campos "Artist Display Name" y "Artist Role" que son los identificadores de artista en la tabla data_artist

In [ ]:
data_artist_columns_sin_key =  ['Artist Prefix', 'Artist Display Bio', 'Artist Suffix', 'Artist Alpha Sort',
       'Artist Nationality', 'Artist Begin Date', 'Artist End Date',
       'Artist Gender', 'Artist ULAN URL', 'Artist Wikidata URL']
data_object = data.drop(data_artist_columns_sin_key, axis = 'columns')
print(data_object.shape)
data_object.columns

Cuando eliminamos los duplicados, la tabla de artistas pasó de tener 4743 registros a 1615.

Esto resulta en una mejora de performance en cuanto a espacio, y facilita mantener la consistencia de los valores en los registros. Como desventaja, vamos a necesitar combinar ambas tablas para poder responder algunas preguntas que involucen relaciones entre columnas en tablas distintas.



---
<a id="section_repaso_joins"></a> 

## Repaso de joins con pandas

[volver a TOC](#section_toc)

`merge` `concat` `join` `append`

Como ejercicio, hagamos ahora el camino inverso, a partir de dos tablas (objetos y artistas) combinemoslas para obtener un único DataFrame con el conjunto total de datos.

Como vimos en la práctica guiada 3, los datos contenidos en objetos pandas pueden combinarse usando los métodos

* `pandas.merge`: combina filas de dos DataFrames en base a una o más claves. Es análogo al join de SQL.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
    
* `pandas.concat`: concatena o apila objetos pandas sobre alguno de los ejes

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

* `pandas.join`: agrega columnas de otro DataFrame

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html

* `pandas.append`: agrega al final las filas de otro DataFrame

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html

**Veamos un ejemplo de `merge`**, comparemos con su tamaño con el DataFrame original (antes de dividirlo en data_objet y data_artist)

In [ ]:
data_all_merge = data_object.merge(data_artist_unique, left_on=['Artist Role', 'Artist Display Name'], 
                             right_on=['Artist Role', 'Artist Display Name'], how="left")

In [ ]:
data_all_merge.shape

In [ ]:
data_all_merge.shape

Estamos definiendo las columnas 'Artist Role', 'Artist Display Name' como claves de los registros para combinarlos en ambos DataFrames.

Dos filas van a ser combinadas en el DataFrame resultado si coinciden en los valores de las claves en los Dataset involucrados en el merge.

`left_on` indica qué columnas del primer DatFrame (el de la izquierda) son la clave en el merge

`right_on` indica qué columnas del segundo DatFrame (el de la derecha) son la clave en el merge

`how` indica como combinar los objetos
* inner: el resultado tiene sólo los registros cuyas claves están en ambos DataFrames
* left: el resultado tiene todos los registros del primer DataFrame (left) aunque la clave pueda no estar en el segundo DataFrame (right)
* rigth: el resultado tiene todos los registros del segundo DataFrame (right) aunque la clave pueda no estar en el primer DataFrame (left)



Veamos un ejemplo de `concat`, axis = 1 indica que concatena las columnas de los dos DataFrame. Observemos que no combina las columnas repetidas sino que el DataFrame resultado tiene una proveniente de cada DataFrame. 

In [ ]:
data_all_concat = pd.concat([data_object, data_artist_unique],  axis = 1)
print(data_object.shape)
print(data_artist_unique.shape)
print(data_all_concat.shape)

data_all_concat.columns[data_all_concat.columns == 'Artist Display Name']

**Veamos un ejemplo de `concat`**, axis = 0 que indica que concatena las filas de los dos DataFrame. Observemos que no es necesario que el orden de las columnas sea el mismo en ambos DataFrames

In [ ]:
data_object_1 = data_object.iloc[0:10, :]

columns_reverse = data_object.columns[::-1]
data_object_2 = data_object.iloc[10:20].loc[:, columns_reverse]

print(data_object_1.shape)
print(data_object_2.shape)

In [ ]:
data_object_1_2 = pd.concat([data_object_1, data_object_2],  axis = 0)
print(data_object_1_2.shape)
data_object_1_2.sample(5)

**Veamos un ejemplo de `join`**. Este método es muy eficiente (mejor que merge) porque usa los índices de los DataFrames. 

Para eso vamos a setear la columnas 'Artist Role' y 'Artist Display Name' como índice en los dos objetos.

Previamente debemos eliminar los registros que sean nulos en alguno de los dos campos de los DataFrames.

Los índices pueden ser creados sobre DataFrames con registros nulos en esos campos, pero el método join da error en esas condiciones.


In [ ]:
print("cantidad de nulos en el campo 'Artist Role' de data_object_index", data_object['Artist Role'].isnull().sum())
print("cantidad de nulos en el campo 'Artist Role' de data_artist_unique_index", data_artist_unique['Artist Role'].isnull().sum())
print("cantidad de nulos en el campo 'Artist Display Name' de data_object_index", data_object['Artist Display Name'].isnull().sum())
print("cantidad de nulos en el campo 'Artist Display Name' de data_artist_unique_index", data_artist_unique['Artist Display Name'].isnull().sum())


In [ ]:
data_artist_notnull = data_artist_unique.dropna(subset=['Artist Role', 'Artist Display Name'], how='any', axis=0)
data_object_notnull = data_object.dropna(subset=['Artist Role', 'Artist Display Name'], how='any', axis=0)

In [ ]:
data_artist_notnull_index = data_artist_notnull.set_index(['Artist Role', 'Artist Display Name'])
data_object_notnull_index = data_object_notnull.set_index(['Artist Role', 'Artist Display Name'])

In [ ]:
data_all_join = data_object_notnull_index.join(data_artist_notnull_index, lsuffix = "object_", rsuffix = "artist_")
print(data_all_join.shape)
data_all_join.sample(3)

Para que el índice se transforme en dos columnas del DataFrame usamos el método `reset_index`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html

In [ ]:
data_all_join = data_all_join.reset_index()
data_all_join.sample(3)

**Veamos un ejemplo de `append`**, que es similar a `concat`

In [ ]:
data_object_1 = data_object.iloc[0:10, :]

columns_reverse = data_object.columns[::-1]
data_object_2 = data_object.iloc[10:20].loc[:, columns_reverse]

print(data_object_1.shape)
print(data_object_2.shape)

In [ ]:
data_object_1_2 = data_object_1.append(data_object_2)
print(data_object_1_2.shape)
data_object_1_2.sample(5)

<a id="section_referencias"></a> 
## Referencias

[volver a TOC](#section_toc)


Python for Data Analysis. Wes McKinney. Cap 8.2

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html